In [1]:
import pandas as pd
import sqlite3
import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

from sqlalchemy import create_engine

In [2]:
#connection to a PostgreSQL local database
# ===========================================

# user_name_var = input('What is the username of your database? ')
host_var = input('What is the host address of your database?')
pw = input("Enter the database password: ")
# db_name = input("Enter the name of the database: ")

string_output = "postgresql://postgres"+':'+pw+'@'+host_var+":5432"+'/earthquake_db'
engine = create_engine(string_output)
connection = engine.connect()
print("Connected to database...")

Connected to database...


In [3]:
#create and connect to a sqlite3 db
conn = sqlite3.connect(r'/Users/rosaicelaroman/Desktop/portfolio_work/earthquake_modeling/Database/db/earthquake_db')

Adding 1 month of data

In [4]:
page_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"

df = pd.read_csv(page_url)
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-04-20T04:14:28.930Z,33.446667,-116.638500,4.61,0.59,ml,22.0,90.00,0.050660,0.1000,...,2022-04-20T04:18:00.424Z,"12km SSE of Anza, CA",earthquake,0.19,0.25,0.180000,17.0,automatic,ci,ci
1,2022-04-20T04:09:52.820Z,19.290667,-155.772003,-0.30,1.87,md,30.0,98.00,NaN,0.3600,...,2022-04-20T04:13:14.240Z,"20 km SSE of Honaunau-Napoopoo, Hawaii",earthquake,0.51,0.71,0.720000,9.0,automatic,hv,hv
2,2022-04-20T04:06:44.830Z,32.886667,-115.503167,3.32,1.74,ml,12.0,93.00,0.050610,0.3400,...,2022-04-20T04:10:29.384Z,"8km NE of Imperial, CA",earthquake,0.80,1.09,0.162000,22.0,automatic,ci,ci
3,2022-04-20T03:58:46.670Z,32.904167,-115.511167,3.49,2.19,ml,18.0,83.00,0.041770,0.3000,...,2022-04-20T04:17:13.213Z,"8km NE of Imperial, CA",earthquake,0.60,0.80,0.196000,29.0,automatic,ci,ci
4,2022-04-20T03:57:31.685Z,38.195900,-117.784800,4.00,1.50,ml,12.0,121.19,0.133000,0.1816,...,2022-04-20T04:01:19.734Z,"35 km SE of Mina, Nevada",earthquake,NaN,2.00,0.150000,8.0,automatic,nn,nn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9528,2022-03-21T04:48:46.720Z,38.823833,-122.808000,3.90,0.43,md,14.0,98.00,0.001678,0.0400,...,2022-03-21T20:17:52.874Z,"7km NW of The Geysers, CA",earthquake,0.39,0.28,0.151000,3.0,reviewed,nc,nc
9529,2022-03-21T04:46:01.380Z,38.821835,-122.760834,1.85,0.85,md,7.0,127.00,0.011170,0.0100,...,2022-03-21T05:29:21.342Z,"3km W of Cobb, CA",earthquake,0.48,0.96,NaN,1.0,automatic,nc,nc
9530,2022-03-21T04:41:58.710Z,19.870000,-155.577000,20.18,1.63,md,24.0,120.00,NaN,0.1500,...,2022-03-22T00:38:03.450Z,"19 km SSE of Waimea, Hawaii",earthquake,0.46,0.99,0.124445,15.0,reviewed,hv,hv
9531,2022-03-21T04:39:20.420Z,38.801666,-122.815498,3.22,0.35,md,7.0,161.00,0.012880,0.0100,...,2022-03-21T04:56:14.173Z,"6km WNW of The Geysers, CA",earthquake,0.59,1.06,NaN,1.0,automatic,nc,nc


In [5]:
#append the new data from USGS to both databases
#=============
#PostgreSQL
df.to_sql('earthquake_raw', engine, if_exists='append', index = False)
#=============
#SQLite
cur = conn.cursor()
df.to_sql('earthquake_raw', conn, if_exists='append', index=False)

9533

Load from database to update with new rows and remove duplicate rows

In [6]:
#Dataframe from postgresql
model_df = pd.read_sql_table(
    "earthquake_raw",
    con=engine
)
model_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.875000,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290000,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.119000,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460000,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48325,2022-03-21T04:48:46.720Z,38.823833,-122.808000,3.90,0.43,md,14.0,98.00,0.001678,0.0400,...,2022-03-21T20:17:52.874Z,"7km NW of The Geysers, CA",earthquake,0.39,0.28,0.151000,3.0,reviewed,nc,nc
48326,2022-03-21T04:46:01.380Z,38.821835,-122.760834,1.85,0.85,md,7.0,127.00,0.011170,0.0100,...,2022-03-21T05:29:21.342Z,"3km W of Cobb, CA",earthquake,0.48,0.96,NaN,1.0,automatic,nc,nc
48327,2022-03-21T04:41:58.710Z,19.870000,-155.577000,20.18,1.63,md,24.0,120.00,NaN,0.1500,...,2022-03-22T00:38:03.450Z,"19 km SSE of Waimea, Hawaii",earthquake,0.46,0.99,0.124445,15.0,reviewed,hv,hv
48328,2022-03-21T04:39:20.420Z,38.801666,-122.815498,3.22,0.35,md,7.0,161.00,0.012880,0.0100,...,2022-03-21T04:56:14.173Z,"6km WNW of The Geysers, CA",earthquake,0.59,1.06,NaN,1.0,automatic,nc,nc


In [7]:
#dataframe from SQLite
model_df2 = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
model_df2

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.875000,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290000,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.119000,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460000,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44892,2022-03-21T04:48:46.720Z,38.823833,-122.808000,3.90,0.43,md,14.0,98.00,0.001678,0.0400,...,2022-03-21T20:17:52.874Z,"7km NW of The Geysers, CA",earthquake,0.39,0.28,0.151000,3.0,reviewed,nc,nc
44893,2022-03-21T04:46:01.380Z,38.821835,-122.760834,1.85,0.85,md,7.0,127.00,0.011170,0.0100,...,2022-03-21T05:29:21.342Z,"3km W of Cobb, CA",earthquake,0.48,0.96,NaN,1.0,automatic,nc,nc
44894,2022-03-21T04:41:58.710Z,19.870000,-155.577000,20.18,1.63,md,24.0,120.00,NaN,0.1500,...,2022-03-22T00:38:03.450Z,"19 km SSE of Waimea, Hawaii",earthquake,0.46,0.99,0.124445,15.0,reviewed,hv,hv
44895,2022-03-21T04:39:20.420Z,38.801666,-122.815498,3.22,0.35,md,7.0,161.00,0.012880,0.0100,...,2022-03-21T04:56:14.173Z,"6km WNW of The Geysers, CA",earthquake,0.59,1.06,NaN,1.0,automatic,nc,nc


In [8]:
#check for duplicates in postgres data
boolean = model_df.duplicated(subset=['id']).any()
print(boolean)

True


In [9]:
#check for duplicates in sqlite data
boolean = model_df2.duplicated(subset=['id']).any()
print(boolean)

False


In [10]:
#drop duplicates in postgres data
model_df.drop_duplicates(subset=['id'], inplace=True)
print(model_df)

                           time   latitude   longitude       depth  mag  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.107500    1.100000  1.3   
1      2022-01-06T01:31:33.698Z  38.181000 -117.818000    7.300000  1.4   
2      2022-01-06T01:24:20.083Z  61.340700 -149.885200   30.000000  1.5   
3      2022-01-06T01:15:15.851Z  62.870300 -150.762500  111.300000  2.1   
4      2022-01-06T01:10:44.629Z  64.014500 -148.765700    1.900000  1.5   
...                         ...        ...         ...         ...  ...   
47799  2022-03-22T17:54:01.161Z  23.392800  121.664100   10.000000  4.8   
47805  2022-03-22T17:26:25.730Z -25.336000 -175.967300   30.460000  4.9   
47968  2022-03-22T06:40:26.308Z -58.347900  -24.315200   10.000000  4.3   
47971  2022-03-22T06:24:44.261Z  26.781800  126.401900   10.000000  4.5   
48323  2022-03-21T05:02:10.609Z  31.547879 -104.154809    6.799976  2.1   

      magType   nst     gap      dmin     rms  ...                   updated  \
0          ml   8.0

In [11]:
#drop duplicates in sqlite data
model_df2.drop_duplicates(subset=['id'], inplace=True)
print(model_df2)

                           time   latitude   longitude   depth   mag magType  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.107500    1.10  1.30      ml   
1      2022-01-06T01:31:33.698Z  38.181000 -117.818000    7.30  1.40      ml   
2      2022-01-06T01:24:20.083Z  61.340700 -149.885200   30.00  1.50      ml   
3      2022-01-06T01:15:15.851Z  62.870300 -150.762500  111.30  2.10      ml   
4      2022-01-06T01:10:44.629Z  64.014500 -148.765700    1.90  1.50      ml   
...                         ...        ...         ...     ...   ...     ...   
44892  2022-03-21T04:48:46.720Z  38.823833 -122.808000    3.90  0.43      md   
44893  2022-03-21T04:46:01.380Z  38.821835 -122.760834    1.85  0.85      md   
44894  2022-03-21T04:41:58.710Z  19.870000 -155.577000   20.18  1.63      md   
44895  2022-03-21T04:39:20.420Z  38.801666 -122.815498    3.22  0.35      md   
44896  2022-03-21T04:33:48.020Z  33.679500 -116.709333   16.00  0.69      ml   

        nst     gap      dmin     rms  

In [12]:
#replace the table in postgres with one without duplicates
model_df.to_sql('earthquake_raw', engine, if_exists='replace', index = False)

897

In [13]:
#replace the table in sqlite with one without duplicates
model_df2.to_sql('earthquake_raw', conn, if_exists='replace', index = False)

44897

In [14]:
#copy data as csv as backup method
model_df.to_csv("../Database/csv/earthquake_raw.csv",index=False)

In [13]:
# Import API key
import sys
weather_api_key = "YOUR KEY HERE!"

if weather_api_key == "YOUR KEY HERE!":
    sys.path.append(r"/Users/rosaicelaroman/Desktop/Data_BootCamp/APIs/config_keys/")
    from api_keys_v2 import weather_api_key

In [17]:
#create new dataframe for locatiions
# location_df = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
# location_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.875000,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.29000,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.119000,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.46000,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38792,2022-04-01T04:48:08.310Z,38.814335,-122.792664,-0.17,0.84,md,9.0,74.00,0.006684,0.0200,...,2022-04-01T05:18:12.579Z,"5km NW of The Geysers, CA",earthquake,0.26,0.85,NaN,1.0,automatic,nc,nc
38793,2022-04-01T04:40:57.610Z,38.821167,-122.788170,1.45,1.14,md,17.0,37.00,0.009826,0.0300,...,2022-04-01T05:25:10.615Z,"6km NNW of The Geysers, CA",earthquake,0.25,0.40,0.49000,3.0,automatic,nc,nc
38794,2022-04-01T04:21:57.850Z,36.900501,-120.989502,5.11,2.29,md,39.0,156.00,0.141500,0.1500,...,2022-04-01T05:33:12.669Z,"22km SW of Los Banos, CA",earthquake,0.51,1.29,0.12000,36.0,automatic,nc,nc
38795,2022-04-01T04:05:26.545Z,-10.369600,-78.346100,66.91,4.30,mb,NaN,158.00,2.182000,0.9400,...,2022-04-01T04:31:09.040Z,near the coast of central Peru,earthquake,11.90,8.80,0.11900,22.0,reviewed,us,us


In [18]:
#create new df with only necessary columns
# cleaned_df = location_df.drop(columns=['nst','rms','magNst','net', 'time', 'magType', 'gap','dmin', 'updated', 'place', 'type', 'horizontalError', 'depthError', 'magError', 'status', 'locationSource', 'magSource'])
# cleaned_df

,latitude,longitude,depth,mag,id
0,37.384000,-117.107500,1.10,1.30,nn00831163
1,38.181000,-117.818000,7.30,1.40,nn00831161
2,61.340700,-149.885200,30.00,1.50,ak0229xypaw
3,62.870300,-150.762500,111.30,2.10,ak0229xwriz
4,64.014500,-148.765700,1.90,1.50,ak0229xvr66
...,...,...,...,...,...
38792,38.814335,-122.792664,-0.17,0.84,nc73712961
38793,38.821167,-122.788170,1.45,1.14,nc73712951
38794,36.900501,-120.989502,5.11,2.29,nc73712941
38795,-10.369600,-78.346100,66.91,4.30,us7000gyvi


http://api.openweathermap.org/geo/1.0/reverse?lat={lat}&lon={lon}&limit={limit}&appid={API key}

In [19]:
#create new columns for data from API
# cleaned_df['Location Name']=""
# cleaned_df['City']=""
# cleaned_df['State']=""
# cleaned_df['Country']=""
# cleaned_df

,latitude,longitude,depth,mag,id,Location Name,City,State,Country
0,37.384000,-117.107500,1.10,1.30,nn00831163,,,,
1,38.181000,-117.818000,7.30,1.40,nn00831161,,,,
2,61.340700,-149.885200,30.00,1.50,ak0229xypaw,,,,
3,62.870300,-150.762500,111.30,2.10,ak0229xwriz,,,,
4,64.014500,-148.765700,1.90,1.50,ak0229xvr66,,,,
...,...,...,...,...,...,...,...,...,...
38792,38.814335,-122.792664,-0.17,0.84,nc73712961,,,,
38793,38.821167,-122.788170,1.45,1.14,nc73712951,,,,
38794,36.900501,-120.989502,5.11,2.29,nc73712941,,,,
38795,-10.369600,-78.346100,66.91,4.30,us7000gyvi,,,,


In [14]:
location_df = pd.read_sql_query("SELECT * FROM earthquake_raw_locations", conn)
location_df

,latitude,longitude,depth,mag,id,Location Name,City,State,Country
0,37.384000,-117.107500,1.10,1.30,nn00831163,Nye County,Nye County,Nevada,US
1,38.181000,-117.818000,7.30,1.40,nn00831161,Esmeralda County,Esmeralda County,Nevada,US
2,61.340700,-149.885200,30.00,1.50,ak0229xypaw,Matanuska-Susitna,Matanuska-Susitna,Alaska,US
3,62.870300,-150.762500,111.30,2.10,ak0229xwriz,Matanuska-Susitna,Matanuska-Susitna,Alaska,US
4,64.014500,-148.765700,1.90,1.50,ak0229xvr66,Denali,Denali,Alaska,US
...,...,...,...,...,...,...,...,...,...
38781,35.322667,-117.783333,4.70,1.09,ci39983311,,,,
38782,63.390200,-151.341800,0.00,1.80,ak02246jyds7,,,,
38783,60.602200,-150.643000,19.40,2.10,ak02246jxbq8,,,,
38784,35.965500,-117.677167,2.37,0.79,ci39983287,,,,


In [15]:
base_url = "http://api.openweathermap.org/geo/1.0/reverse?"



http://api.openweathermap.org/geo/1.0/reverse?lat=51.5098&lon=-0.1180&limit=5&appid={API key}

In [16]:
response = requests.get(base_url+"lat="+'37.3840'+'&lon='+'-117.107500'+'&limit='+'1'+'&appid=' + weather_api_key).json()
print(response[0]['country'])

US


In [23]:
import requests
import time

for index, row in location_df.iterrows():
    # count+=1
    lat = row['latitude']
    lon = row['longitude']
    loc_found = []

    if row['Country'] != "":
        continue

    print(f'Retrieving results for Index {index}: {lat}')
    try:
        response = requests.get(base_url+"lat="+str(lat)+'&lon='+str(lon)+'&limit='+'1'+'&appid=' + weather_api_key).json()
        # response = requests.get(base_url+city+"&units="+units+'&appid=' + weather_api_key).json()
        location_df.loc[index, 'Location Name'] = response[0]['name']
        location_df.loc[index, 'City'] = response[0]['local_names']['en']
        location_df.loc[index, 'State'] = response[0]['state']
        location_df.loc[index, 'Country'] = response[0]['country']
        loc_found.append(lat)

        print(f'Found lat: {lat}')
    except:
        print(f"Missing lat: {lat}... Skipping")
        location_df.loc[index, 'Location Name'] = float('NaN')
        location_df.loc[index, 'City'] = float('NaN')
        location_df.loc[index, 'State'] = float('NaN')
        location_df.loc[index, 'Country'] = float('NaN')
    print(f'--------------------')
    time.sleep(2)


Retrieving results for Index 34128: 19.2121658325195
Found lat: 19.2121658325195
--------------------
Retrieving results for Index 34129: 36.0283333
Found lat: 36.0283333
--------------------
Retrieving results for Index 34130: 19.1988334655762
Found lat: 19.1988334655762
--------------------
Retrieving results for Index 34131: 36.0285
Found lat: 36.0285
--------------------
Retrieving results for Index 34132: 64.0423
Found lat: 64.0423
--------------------
Retrieving results for Index 34133: 19.1698341369629
Found lat: 19.1698341369629
--------------------
Retrieving results for Index 34134: 38.7736667
Found lat: 38.7736667
--------------------
Retrieving results for Index 34135: 37.8398
Missing lat: 37.8398... Skipping
--------------------
Retrieving results for Index 34136: 26.9646
Found lat: 26.9646
--------------------
Retrieving results for Index 34137: 27.026
Found lat: 27.026
--------------------
Retrieving results for Index 34138: 60.2539
Found lat: 60.2539
-------------------

In [24]:
location_df

,latitude,longitude,depth,mag,id,Location Name,City,State,Country
0,37.384000,-117.107500,1.10,1.30,nn00831163,Nye County,Nye County,Nevada,US
1,38.181000,-117.818000,7.30,1.40,nn00831161,Esmeralda County,Esmeralda County,Nevada,US
2,61.340700,-149.885200,30.00,1.50,ak0229xypaw,Matanuska-Susitna,Matanuska-Susitna,Alaska,US
3,62.870300,-150.762500,111.30,2.10,ak0229xwriz,Matanuska-Susitna,Matanuska-Susitna,Alaska,US
4,64.014500,-148.765700,1.90,1.50,ak0229xvr66,Denali,Denali,Alaska,US
...,...,...,...,...,...,...,...,...,...
38781,35.322667,-117.783333,4.70,1.09,ci39983311,Kern County,Kern County,California,US
38782,63.390200,-151.341800,0.00,1.80,ak02246jyds7,Denali,Denali,Alaska,US
38783,60.602200,-150.643000,19.40,2.10,ak02246jxbq8,Kenai Peninsula,Kenai Peninsula,Alaska,US
38784,35.965500,-117.677167,2.37,0.79,ci39983287,Inyo County,Inyo County,California,US


In [25]:
#replace the table in sqlite with one without duplicates
location_df.to_sql('earthquake_raw_locations', conn, if_exists='replace', index = False)

#replace the table in postgres with one without duplicates
location_df.to_sql('earthquake_raw_locations', engine, if_exists='replace', index = False)

786

In [28]:
# Close connection to SQLite database
conn.close()
connection.close()